<a href="https://colab.research.google.com/github/YangchenHuang/Algorithmic_Trading/blob/master/Trading%20Bot%20with%20RL%20%26%20ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Use Google Colab as the development environment
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=6e20b134d9a6207e4eff2c0cecdf60a5dc57d7a5f0b87236a595d0e95c97fba3
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [0]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
from bayes_opt import BayesianOptimization

In [0]:
# Get the data from github dataset
!rm -rf Algorithmic_Trading
!git clone https://github.com/YangchenHuang/Algorithmic_Trading

Cloning into 'Algorithmic_Trading'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 139 (delta 21), reused 0 (delta 0), pack-reused 90
Receiving objects: 100% (139/139), 26.02 MiB | 8.48 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [0]:
df = pd.read_csv('./Algorithmic_Trading/dataset/NVDA.csv')
df.head()

,PERMNO,date,VOL,NUMTRD,PRC,OPENPRC
0,86580,1/3/2007,19273722,67875,24.053333,24.713333
1,86580,1/4/2007,15957279,53389,23.940000,23.966667
2,86580,1/5/2007,20729060,75741,22.440000,23.373333
3,86580,1/8/2007,10955217,43714,22.606667,22.520000
4,86580,1/9/2007,12750524,48630,22.166667,22.640000


In [0]:
# Data Cleaning, create two list
data = df[['PRC', 'VOL', 'OPENPRC']].to_numpy().flatten()
data = data.tolist()
data = [abs(_) for _ in data]
close = df['PRC'].tolist()
close = [abs(_) for _ in close]
# Train/Test Data Split
l = len(close) - 1
test_size=l//2
dev_size=0

In [0]:
# This function is to get the historical window of data as the state at time t
def get_state(data, t, n):
    d = t - n + 1
    block = data[d*3 : (t + 1)*3] if d >= 0 else -d * [data[0], data[1], data[2]] + data[0 : (t + 1)*3]
    res = []
    for i in range(n - 1):
        for j in range(3):
            if block[i*3+j]==0:
                res.append(0)
            else:
                res.append((block[(i + 1)*3+j] - block[i*3+j])/block[i*3+j])
    return np.array([res])

In [0]:
class Deep_Evolution_Strategy:

    inputs = None

    def __init__(
        self, weights, reward_function, population_size, sigma, learning_rate, trade_train, buy, model
    ):
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate
        self.fsigma = sigma
        self.trade_train = trade_train
        self.buy = buy
        self.model = model
        self.wc = weights
    
    # Random Mutation
    def _get_weight_from_population(self, weights, population):
        weights_population = []
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        return weights_population

    def get_weights(self):
        return self.weights

    # Training part
    def train(self, epoch = 100, print_every = 1):
        lasttime = time.time()
        reward_list=[]
        max_validate=0
        best_model=self.model
        for i in range(epoch):
            population = []
            rewards = np.zeros(self.population_size)
            # Randomly generate populations
            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            # Get reward for each one in the population
            for k in range(self.population_size):
                weights_population = self._get_weight_from_population(
                    self.weights, population[k]
                )
                rewards[k] = self.reward_function(weights_population)
            # Softmax Weighted reward
            e_rewards = np.exp(rewards - np.max(rewards))
            e_rewards = e_rewards / e_rewards.sum(axis=0)
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.wc[index] = (
                    w
                    + self.learning_rate
                    * np.dot(A.T, e_rewards).T
                )
            now_reward=self.reward_function(self.wc)
            # Mutation rate heauristic
            if i==0:
                final_reward = now_reward 
            if i>0:
                final_reward = reward_list[i-1]
            if i>0 and (self.sigma > self.fsigma/4 or now_reward > reward_list[i-1]):
                self.weights = self.wc
                final_reward = now_reward
            if i>0:
                if final_reward<=0:
                    self.sigma = self.fsigma
                if self.sigma > self.fsigma/2:
                    if final_reward>max(reward_list[max(0, i-5):i]):
                        self.sigma=self.sigma*np.clip((final_reward-max(reward_list))/(final_reward+1e-10),0.95, 0.999)
                    if final_reward<np.average(reward_list):
                        self.sigma=min(self.sigma*np.clip(np.average(reward_list)/(np.average(reward_list)-final_reward+1e-10),1.001, 1.05),self.fsigma)
                elif self.sigma > self.fsigma/4:
                    if final_reward>max(reward_list[max(0, i-10):i]):
                        self.sigma=self.sigma*np.clip((final_reward-max(reward_list))/(final_reward+1e-10),0.9, 0.95)
                elif self.sigma > self.fsigma/10:
                    if final_reward>max(reward_list):
                        self.sigma=self.sigma*(np.random.random()*0.4+0.5)
                else:
                    if final_reward>max(reward_list):
                        self.sigma=self.sigma*(np.random.random()*0.5)
            # print(self.sigma)
            reward_list.append(final_reward)
            # v=self.validate()
            # if (i + 1) % print_every == 0:
            #     print(
            #         'iter %d. reward: %f validate: %f %%'
            #         % (i + 1, final_reward, v)
            #     )
            if self.sigma<10e-5:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, final_reward)
                )
                self.trade_train(self.weights)
                self.buy(self.weights)
                break
            if (i + 1) % print_every == 0:
                print(
                    'iter %d. reward: %f'
                    % (i + 1, final_reward)
                )
            # if v > max_validate:
            #     best_model.set_weights=self.weights
            # if i==epoch-1:
            #     self.buy()
            
        print('time taken to train:', time.time() - lasttime, 'seconds')

# Neural Network Structure
class Model:
    def __init__(self, input_size, layer_size, output_size):
        self.weights = [
            np.random.randn(input_size, layer_size)*0.01,
            np.random.randn(layer_size, output_size)*0.01,
            np.random.randn(layer_size+1, 1)*0.01,
            np.zeros((1, layer_size)),
            np.zeros((1, output_size)),
            np.zeros((1,1))
        ]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-3]
        a1=np.tanh(feed)
        # Buy/Sell Signal
        decision = np.dot(a1, self.weights[1]) + self.weights[-2]
        z = np.append(np.argmax(decision[0]), a1)
        # Trade Shares
        trade = np.dot(z, self.weights[2]) + self.weights[-1]
        trade = trade[0][0]
        if np.isnan(trade):
            trade = 0
        trade = max(trade, 0)
        return decision, trade

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

In [0]:
# Trading Agent
class Agent:
    def __init__(
        self,
        population_size,
        sigma,
        learning_rate,
        model,
        money,
        skip,
        window_size,
    ):
        self.window_size = window_size
        self.skip = skip
        self.POPULATION_SIZE = population_size
        self.SIGMA = sigma
        self.LEARNING_RATE = learning_rate
        self.model = model
        self.initial_money = money
        self.es = Deep_Evolution_Strategy(
            self.model.get_weights(),
            self.get_reward,
            self.POPULATION_SIZE,
            self.SIGMA,
            self.LEARNING_RATE,
            self.trade_train,
            self.buy,
            self.model
        )

    def act(self, sequence):
        decision, buy = self.model.predict(np.array(sequence))
        return np.argmax(decision[0]), int(buy)

    # Reward Function
    def get_reward(self, weights):
        initial_money = self.initial_money
        starting_money = initial_money
        self.model.weights = weights
        state = get_state(data, 30, self.window_size + 1)
        inventory = []
        quantity = 0
        pnl = []
        maxdown = []
        loss_penalty = 1
        risk_penalty = 1
        now_money = starting_money
        # final_money = starting_money
        for t in range(30, l-test_size-dev_size, self.skip):
            action, trade = self.act(state)
            next_state = get_state(data, t + 1, self.window_size + 1)
            max_buy = min(trade, starting_money//close[t]//2, initial_money//close[t])
            if action == 1 and initial_money >= close[t]:
                buy_units = max_buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                sell_units = quantity
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                # final_money = initial_money
            now_money=initial_money+quantity*close[t]
            pnl.append(((now_money - starting_money) / starting_money) * 100)
            if t>=60:
                maxdown.append(max(pnl[t-60:t-31])-pnl[t-31])
            state = next_state
        # Loss penalty heuristic
        if now_money>starting_money and min(pnl)<0:
            loss_penalty = max(1, 1/(1+min(pnl)*0.01))
        # Risk penalty heuristic
        if now_money>starting_money and max(maxdown)>0:
            risk_penalty = max(1, 1/(1-max(maxdown)*0.01))
        return ((now_money - starting_money) / starting_money) * 100/ (loss_penalty * risk_penalty)

    def fit(self, iterations, checkpoint):
        self.es.train(iterations, print_every = checkpoint)

    # Trading Strategy on training set
    def trade_train(self, weights):
        initial_money = self.initial_money
        self.model.weights = weights
        state = get_state(data, 30, self.window_size + 1)
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        quantity = 0
        now_money = initial_money
        pnl=[]
        # final_money = starting_money
        for t in range(30, l-test_size, self.skip):
            action, trade = self.act(state)
            next_state = get_state(data, t + 1, self.window_size + 1)
            max_buy = min(trade, starting_money//close[t]//2, initial_money//close[t])
            if action == 1 and initial_money >= close[t]:
                buy_units = max_buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
                states_buy.append(t)
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                sell_units = quantity
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                # final_money = initial_money
                states_sell.append(t)
            now_money=initial_money+quantity*close[t]
            pnl.append(((now_money - starting_money) / starting_money) * 100)
            state = next_state
        invest = ((now_money - starting_money) / starting_money) * 100
        print(
            '\ntotal gained %f, total investment %f %%'
            % (now_money - starting_money, invest)
        )
        plt.figure(figsize = (20, 10))
        plt.plot(close, label = 'true close', c = 'g')
        plt.plot(
            close, 'X', label = 'predict buy', markevery = states_buy, c = 'b'
        )
        plt.plot(
            close, 'o', label = 'predict sell', markevery = states_sell, c = 'r'
        )
        plt.legend()
        plt.figure(figsize = (20, 10))
        plt.plot(pnl, label = 'pnl', c = 'r')
        plt.show()


    # Trading Strategy on test set
    def buy(self, weights):
        initial_money = self.initial_money
        self.model.weights = weights
        state = get_state(data, l-test_size, self.window_size + 1)
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        quantity = 0
        now_money = initial_money
        pnl=[]
        # final_money = starting_money
        for t in range(l-test_size, l, self.skip):
            action, trade = self.act(state)
            next_state = get_state(data, t + 1, self.window_size + 1)
            max_buy = min(trade, starting_money//close[t]//2, initial_money//close[t])
            if action == 1 and initial_money >= close[t]:
                buy_units = max_buy
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
                states_buy.append(t)
                print(
                    'day %d: buy %d units at price %f, total balance %f'
                    % (t, buy_units, total_buy, initial_money)
                )
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                sell_units = quantity
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                # final_money = initial_money
                states_sell.append(t)
                print(
                    'day %d, sell %d units at price %f, total balance %f,'
                    % (t, sell_units, total_sell, initial_money)
                )
            now_money=initial_money+quantity*close[t]
            pnl.append(((now_money - starting_money) / starting_money) * 100)
            state = next_state
        invest = ((now_money - starting_money) / starting_money) * 100
        print(
            '\ntotal gained %f, total investment %f %%'
            % (now_money - starting_money, invest)
        )
        plt.figure(figsize = (20, 10))
        plt.plot(close, label = 'true close', c = 'g')
        plt.plot(
            close, 'X', label = 'predict buy', markevery = states_buy, c = 'b'
        )
        plt.plot(
            close, 'o', label = 'predict sell', markevery = states_sell, c = 'r'
        )
        plt.legend()
        plt.figure(figsize = (20, 10))
        plt.plot(pnl, label = 'pnl', c = 'r')
        plt.show()

In [0]:
def best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    model = Model(window_size*3, size_network, 3)
    agent = Agent(
        population_size,
        sigma,
        learning_rate,
        model,
        100000,
        skip,
        window_size,
    )
    agent.fit(5, 5)
    return agent.es.reward_function(agent.es.weights)

In [0]:
# Used for Bayesian Optimization
def find_best_agent(
    window_size, skip, population_size, sigma, learning_rate, size_network
):
    global accbest
    param = {
        'window_size': int(np.around(window_size)),
        'skip': int(np.around(skip)),
        'population_size': int(np.around(population_size)),
        'sigma': max(min(sigma, 1), 0.0001),
        'learning_rate': max(min(learning_rate, 0.5), 0.000001),
        'size_network': int(np.around(size_network)),
    }
    print('\nSearch parameters %s' % (param))
    investment = best_agent(**param)
    print('stop after 100 iteration with investment %f' % (investment))
    if investment > accbest:
        costbest = investment
    return investment

In [0]:
# Bayesian Optimization to find the best-in-class hyperparameters
accbest = 0.0
NN_BAYESIAN = BayesianOptimization(
    find_best_agent,
    {
        'window_size': (7, 30),
        'skip': (1, 1),
        'population_size': (5, 25),
        'sigma': (0.2, 0.5),
        'learning_rate': (0.2, 0.5),
        'size_network': (100, 1000),
    },
)
NN_BAYESIAN.maximize(init_points = 30, n_iter = 20, acq = 'ei', xi = 0.0)

In [0]:
# Reinforcement Learning Using manually selected hyperparameters
model = Model(input_size = 30*3, 
              layer_size = 500, 
              output_size = 3)
agent = Agent(population_size = 15, 
              sigma = 0.5, 
              learning_rate = 0.5, 
              model = model, 
              money = 100000,  
              skip = 1, 
              window_size = 30)
agent.fit(500, 5)

In [0]:
# Reinforcement Learning Using Bayesian selected hyperparameters
model = Model(input_size = int(np.around(NN_BAYESIAN.max['params']['window_size']))*3, 
              layer_size = int(np.around(NN_BAYESIAN.max['params']['size_network'])), 
              output_size = 3)
agent = Agent(population_size = int(np.around(NN_BAYESIAN.max['params']['population_size'])), 
              sigma = NN_BAYESIAN.max['params']['sigma'], 
              learning_rate = NN_BAYESIAN.max['params']['learning_rate'], 
              model = model, 
              money = 100000, 
              skip = int(np.around(NN_BAYESIAN.max['params']['skip'])), 
              window_size = int(np.around(NN_BAYESIAN.max['params']['window_size'])))
agent.fit(500, 5)